<a href="https://colab.research.google.com/github/lapshinaaa/recsys-tasks/blob/main/RecSys2_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Recommender Systems — Notebook #2</center></h1>

<center>
<img src="https://avatars.mds.yandex.net/get-grocery-goods/2783132/ab847ff6-95e3-4c4e-831a-0576d1949a9e/orig" width="300" />
</center>

**In this notebook, we will work through the following:**

- Explore the dataset of user interaction events from the **Yandex Lavka** application.
- Review the course competition (Kaggle contest):  
  https://www.kaggle.com/t/eb7d5a01648e4e7cb0dfa404d29497ea
- Implement a **baseline recommender model**.
- Train more advanced models (e.g., **CatBoost** for ranking).
- Implement several **new ranking quality metrics**.

In [ ]:
# !pip install catboost

# !pip install numpy==1.23.5

In [ ]:
import zipfile
import requests

import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
from textwrap import wrap
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import roc_auc_score, log_loss, ndcg_score

# 🗄 Dataset:

In [ ]:
def download_and_extract(url: str, filename: str, chunk_size: int = 1024):
    # load the file
    response = requests.get(url, stream=True)
    response.raise_for_status()

    total_size = int(response.headers.get('content-length', 0))

    # write the file
    with open(filename, "wb") as f:
        with tqdm(
            total=total_size,
            unit='B',
            unit_scale=True,
            desc=filename,
            bar_format='{l_bar}{bar:50}{r_bar}{bar:-50b}'  # формат для красоты
        ) as pbar:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

    # inzip archive
    with zipfile.ZipFile(filename, "r") as zip_ref:
        print(f"\nРаспаковываем {filename}...")
        zip_ref.extractall(".")
        print(f"Файлы из {filename} успешно извлечены\n")

In [ ]:
download_and_extract(
    url="https://www.kaggle.com/api/v1/datasets/download/thekabeton/ysda-recsys-2025-lavka-dataset",
    filename="lavka.zip"
)

In [ ]:
train = pl.read_parquet('train.parquet')
test = pl.read_parquet('test.parquet')

# for kaggle: train = train.sample(200000, shuffle=True)

train.head(100)

In [ ]:
test.head(5)

# 👀 Taking a look at the dataset

In [ ]:
train.group_by(
    "action_type"
).agg(
    pl.len().alias("total_actions")
)

In [ ]:
city_analysis = train.group_by("city_name").agg(
    pl.len().alias("actions_count")
).sort("actions_count", descending=True)

plt.figure(figsize=(10, 6))
sns.barplot(
    x="city_name",
    y="actions_count",
    data=city_analysis.to_pandas()
)
plt.title("Распределение просмотров по городам")
plt.xlabel("Город")
plt.ylabel("Количество просмотров")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Посмотрим на самые популярные покупки:

In [ ]:
top_10 = train.filter(
    pl.col('action_type') == 'AT_Purchase'
).group_by(
    'product_name'
).agg(
    pl.len().alias("total_purchase"),
    pl.col('product_image').first()
).sort(
    'total_purchase', descending=True
).head(10)

top_10

In [ ]:
def load_poster(row):
    title, poster_url = row[0], row[2]
    try:
        response = requests.get(poster_url, timeout=20)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)), title

    except Exception as e:
        print(f"Error loading poster for '{title}': {e}")
        return None, None

def show_posters(data):
    fig, axes = plt.subplots(2, 5, figsize=(20, 10))
    plt.subplots_adjust(hspace=0.5, wspace=0.3)

    rows = [row for row in data.iter_rows()]

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(load_poster, rows))

    for idx, (img, title) in enumerate(results):
        ax = axes[idx//5, idx%5]
        if img and title:
            ax.imshow(img)
            wrapped_title = "\n".join(wrap(title, width=40))
            ax.set_title(wrapped_title)
        else:
            ax.set_title("Image not available", fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
show_posters(top_10)

# 🎲 Рандомный сабмит:

In [ ]:
random_submit = test.select(
    'index',
    'request_id'
).sample(
    fraction=1.0,
    shuffle=True
)

# random_submit.write_csv('random_submit.csv')

random_submit

# 📈 Бейзлайн:

In [ ]:
count_purchase_in_train = train.filter(
    pl.col('action_type') == "AT_Purchase"
).group_by(
    'user_id',
    'product_id'
).agg(
    pl.len()
)

count_purchase_in_train

In [ ]:
baseline_submit = test.join(
    count_purchase_in_train,
    on=["user_id", "product_id"],
    how="left"
).with_columns(
    pl.col("len").fill_null(0)
).sort(
    'len',
    descending=True
).select(
    'index',
    'request_id'
)

# baseline_submit.write_csv('baseline_submit.csv')

baseline_submit

# 🦾 CatBoost

<center><img src="Timesplit1.svg" width="1100" /></center>


Давайте соберём какие-то фичи из данных и обучим на них градиентный бустинг. Нужно не забывать про временные лики. Нельзя давать модели видеть данные из будущего, поэтому фичи для каждого семпла должны быть посчитаны на данных из прошлого. В простейшей схеме предлагается разделить размеченые данные на 3 части:
- Вторая часть - train
- Третья часть - validation
- Первую часть используем для расчёта статистик для трейна
- Для валидации считаем статистики используя первую и вторую части вместе

#### Делим train на 3 части:

In [ ]:
train_len_div3 = int(len(train) / 3)

train = train.sort(
    'timestamp'
)

train_part1 = train[:train_len_div3]
train_part2 = train[train_len_div3:train_len_div3 * 2]
train_part3 = train[train_len_div3 * 2:]

Посчитаем количество покупок каждого товара для каждого пользователя на первой части трейна:

In [ ]:
def calculate_count_purchase_by_user_and_product(dataset: pl.DataFrame) -> pl.DataFrame:
    count_purchase_by_user_and_product = dataset.filter(
        pl.col('action_type') == "AT_Purchase"
    ).group_by(
        'user_id',
        'product_id'
    ).agg(
        pl.len().alias('count_purchase_by_user_and_product')
    )

    return count_purchase_by_user_and_product

count_purchase_by_user_and_product_for_train = calculate_count_purchase_by_user_and_product(train_part1)

assert set(count_purchase_by_user_and_product_for_train.columns) == set(['user_id', 'product_id', 'count_purchase_by_user_and_product'])

count_purchase_by_user_and_product_for_train.head(5)

Теперь посчитаем CTR товаров по всем юзерам.

CTR (Click-Through Rate) — коэффициент кликабельности, отношение количесва кликов к количеству показов.

В нашем случае - отношение AT_Click к AT_View.

Посчитаем CTR для каждого товара на первой части трейна:

In [ ]:
def calculate_ctr(dataset: pl.DataFrame) -> pl.DataFrame:
    data = train_part1.group_by(
        'action_type',
        'product_id'
    ).agg(
        pl.len()
    )

    clicks = data.filter(
         pl.col('action_type') == "AT_Click"
    )

    views = data.filter(
         pl.col('action_type') == "AT_View"
    )

    ctr = clicks.join(
        views,
        on='product_id'
    ).with_columns(
        ctr=pl.col('len') / pl.col('len_right')
    ).select(
        'product_id',
        'ctr'
    )

    return ctr

ctr_for_train = calculate_ctr(train_part1)

assert set(ctr_for_train.columns) == set(['product_id', 'ctr'])

ctr_for_train.head(5)

Создаём тренировочный пул для катбуста. Берём события из второй части датасета и клеим к ним созданные фичи:

In [ ]:
def join_features_to_dataset(
    dataset: pl.DataFrame,
    count_purchase_by_user_and_product: pl.DataFrame,
    ctr: pl.DataFrame
) -> pl.DataFrame:
    catboost_pool = dataset.filter(
        pl.col('action_type').is_in(["AT_View", "AT_CartUpdate"])
    ).with_columns(
        target=pl.when(pl.col('action_type') == "AT_View").then(0).otherwise(1)
    ).group_by(
        ['product_id', 'request_id']
    ).max().drop(
        'source_type',
        'store_id',
        'timestamp',
        'product_image',
        'product_name',
        'city_name',
        'position_in_request',
        'product_category',
        'action_type'
    ).join(
        ctr,
        on='product_id',
        how='left'
    ).join(
        count_purchase_by_user_and_product,
        on=['user_id', 'product_id'],
        how='left'
    )

    return catboost_pool

train_catboost = join_features_to_dataset(
    train_part2,
    calculate_count_purchase_by_user_and_product(train_part1),
    calculate_ctr(train_part1)
)

assert set(train_catboost.columns) == set(['ctr', 'count_purchase_by_user_and_product', 'target', 'request_id', 'product_id', 'user_id'])

train_catboost.head(5)

Проделываем то-же самое для валидации. Фичи считаем по событиям из 1 и 2 части датасета. Затем клеим их к 3 части:

In [ ]:
train_parts_1_2 = pl.concat([train_part1, train_part2])

val_catboost = join_features_to_dataset(
    train_part3,
    calculate_count_purchase_by_user_and_product(train_parts_1_2),
    calculate_ctr(train_parts_1_2)
)

val_catboost.head(5)

#### Обучаем катбуст:

In [ ]:
from catboost import CatBoostClassifier, Pool

# Пример данных
train_data = Pool(
    data=train_catboost.drop(['target', 'request_id', 'product_id', 'user_id']).to_pandas(),
    label=train_catboost['target'].to_list()
)

val_data = Pool(
    data=val_catboost.drop(['target', 'request_id', 'product_id', 'user_id']).to_pandas(),
    label=val_catboost['target'].to_list()
)

In [ ]:
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.01,
    depth=2,
    loss_function="Logloss",
    eval_metric="AUC",
    early_stopping_rounds=50,
)

In [ ]:
model.fit(
    train_data,
    eval_set=val_data,
    # plot=True
)

In [ ]:
y_pred_proba = model.predict_proba(val_catboost.drop(['target']).to_pandas())[:, 1]

roc_auc = roc_auc_score(val_catboost['target'].to_list(), y_pred_proba)
print(f"ROC AUC: {roc_auc:.4f}")

logloss = log_loss(val_catboost['target'].to_list(), y_pred_proba)
print(f"LogLoss: {logloss:.4f}")

#### Важности фичей:

In [ ]:
for name, fstr in zip(model.feature_names_, model.feature_importances_):
    print(name, ':', fstr)

Переделаем функцию джойна для тестового датасета:

In [ ]:
def join_features_to_val_dataset(
    dataset: pl.DataFrame,
    count_purchase_by_user_and_product: pl.DataFrame,
    ctr: pl.DataFrame
) -> pl.DataFrame:
    catboost_pool = dataset.drop(
        'source_type',
        'store_id',
        'timestamp',
        'city_name',
        'product_name',
        'product_category',
        'product_image'
    ).join(
        ctr,
        on='product_id',
        how='left'
    ).join(
        count_purchase_by_user_and_product,
        on=['user_id', 'product_id'],
        how='left'
    )

    catboost_pool = catboost_pool.drop(
        'user_id',
        'product_id',
        'request_id'
    )

    return catboost_pool

In [ ]:
kaggle_catboost = join_features_to_val_dataset(
    test,
    calculate_count_purchase_by_user_and_product(train),
    calculate_ctr(train)
)

kaggle_catboost.head(5)

In [ ]:
test_data = test['index', 'request_id']

test_data.with_columns(
    predict=model.predict_proba(kaggle_catboost.to_pandas())[:, 1]
).sort(
    'predict',
    descending=True
).select(
    'index',
    'request_id'
).write_csv('cb_submit.csv')

In [ ]:
val_catboost

# 🎯 Метрики качества ранжирования

In [ ]:
import sklearn

catboost_predicts = val_catboost.with_columns(
    predict=model.predict_proba(val_catboost.drop(['target']).to_pandas())[:, 1]
)

true = []
pred = []

for i in catboost_predicts.group_by('request_id'):
    value = i[1].sort('target', descending=True)[:10]
    if sum(value['target']) == 0:
        continue
    l = [0] * (10 - len(value['target']))
    true.append(value['target'].to_list() + l)
    pred.append(value['predict'].to_list() + l)

# Формулы для MAP@K (Mean Average Precision at K)

## 1. **Precision@K**
Доля релевантных документов среди первых `K` результатов:
$$
\text{Precision}@K = \frac{\text{Количество релевантных документов в топ-}K}{K}
$$

---

## 2. **Average Precision@K (AP@K)**
Средняя точность для одного запроса, учитывающая позиции релевантных документов в топ-`K`:
$$
\text{AP}@K = \frac{\sum_{k=1}^{K} \text{Precision}@k \cdot \text{rel}(k)}{\text{Количество релевантных документов в топ-}K}
$$
- `rel(k)` = 1, если документ на позиции `k` релевантен, иначе 0.
- Если в топ-`K` нет релевантных документов, то `AP@K = 0`.

---

## 3. **MAP@K (Mean Average Precision at K)**
Среднее значение AP@K по всем запросам:
$$
\text{MAP}@K = \frac{1}{Q} \sum_{q=1}^{Q} \text{AP}@K^{(q)}
$$
- `Q` — общее количество запросов.
-  $AP@K^{(q)}$ — Average Precision@K для запроса `q`.

In [ ]:
def ap_at_k(y_true, y_pred, k):
    if np.sum(y_true) == 0:
        return 0.0
    sorted_indices = np.argsort(y_pred)[::-1]
    top_k_indices = sorted_indices[:k]
    y_true_k = y_true[top_k_indices]

    cumulative_precision = 0.0
    relevant_seen = 0
    for i in range(len(y_true_k)):
        if y_true_k[i]:
            relevant_seen += 1
            precision_at_i = relevant_seen / (i + 1)
            cumulative_precision += precision_at_i

    return cumulative_precision / relevant_seen

def map_at_k(true_relevance, predicted_scores, k):
    total_ap = 0.0

    for y_true, y_pred in zip(true_relevance, predicted_scores):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        ap = ap_at_k(y_true, y_pred, k)
        total_ap += ap

    return total_ap / len(true_relevance)

custom_map = map_at_k(true, pred, 10)

print(f"MAP@10: {custom_map:.4f}")

# Формулы для NDCG (Normalized Discounted Cumulative Gain)

## 1. **CG (Cumulative Gain)**
Простая сумма релевантностей первых `p` документов в результатах ранжирования:
$$
\text{CG}_p = \sum_{i=1}^{p} \text{rel}_i
$$
- `rel_i` — релевантность документа на позиции `i`.

---

## 2. **DCG (Discounted Cumulative Gain)**
Учитывает порядок документов, дисконтируя релевантность на более низких позициях:
$$
\text{DCG}_p = \sum_{i=1}^{p} \frac{\text{rel}_i}{\log_2(i + 1)}
$$

---

## 3. **IDCG (Ideal DCG)**
Максимально возможный DCG при идеальном порядке документов:
$$
\text{IDCG}_p = \sum_{i=1}^{p} \frac{\text{rel}_i^{\text{(ideal)}}}{\log_2(i + 1)}
$$
где $rel_i^{(ideal)}$ — релевантности документов, отсортированные по убыванию.

---

## 4. **NDCG (Normalized DCG)**
Нормализованная версия DCG в диапазоне [0, 1]:
$$
\text{NDCG}_p = \frac{\text{DCG}_p}{\text{IDCG}_p}
$$

In [ ]:
def ndcg_at_10(true_relevance, predicted_scores):
    ndcg = 0.0

    for true, pred in zip(true_relevance, predicted_scores):
        true = np.array(true)
        pred = np.array(pred)

        top_10_indices = np.argsort(pred)[::-1]
        rels = true[top_10_indices]

        dcg = 0.0
        for i, rel in enumerate(rels, 1):
            dcg += rel / np.log2(i + 1)

        ideal_rels = sorted(true, reverse=True)
        idcg = 0.0
        for i, rel in enumerate(ideal_rels, 1):
            idcg += rel / np.log2(i + 1)

        ndcg += dcg / idcg

    return ndcg / len(true_relevance)

custom_ndcg = ndcg_at_10(true, pred)
sklearn_ndcg = ndcg_score(true, pred, k=10, ignore_ties=True)

print(f"Custom NDCG@10: {custom_ndcg:.4f}")
print(f"Sklearn NDCG@10: {sklearn_ndcg:.4f}")

assert abs(custom_ndcg - sklearn_ndcg) < 1e-4

# Метрика Novelty в рекомендательных системах

**Novelty** (новизна) отражает способность системы рекомендовать элементы, которые **новы** или **неизвестны** пользователю.  
Novelty не требует, чтобы рекомендации были полезными — только **непривычными**. Основной подход к расчету:

---

## **На основе популярности элементов**
Чем менее популярен элемент, тем выше его новизна:
$$
\text{Novelty}(i) = 1 - \text{Popularity}(i)
$$
- `Popularity(i)` — нормированная популярность элемента (например, доля пользователей, взаимодействовавших с `i`).

**Средняя Novelty для списка рекомендаций**:
$$
\text{Novelty}@K = \frac{1}{K} \sum_{i=1}^{K} \left(1 - \text{Popularity}(i)\right)
$$

In [ ]:
total_purchasing_users = (
    train_parts_1_2.filter(pl.col('action_type') == "AT_Purchase")
    ['user_id'].unique().shape[0]
)

product_novelty_df = (
    train_parts_1_2.filter(pl.col('action_type') == "AT_Purchase")
    .group_by(['product_id', 'user_id'])
    .agg()
    .group_by('product_id')
    .agg(
        pl.len().alias('unique_buyers_count')
    )
    .with_columns(
        novelty_score=1 - (pl.col('unique_buyers_count') / total_purchasing_users)
    )
    .drop('unique_buyers_count')
)

total_novelty_score = 0.0
processed_requests_count = 0

predicts_with_novelty = catboost_predicts.join(
    product_novelty_df,
    on='product_id',
    how='left'
).fill_null(1).group_by('request_id')

for request_id, recommendations in predicts_with_novelty:

    top10_recommendations = recommendations.sort('target', descending=True).head(10)
    average_novelty = top10_recommendations['novelty_score'].mean()

    if average_novelty is not None:
        processed_requests_count += 1
        total_novelty_score += average_novelty

final_novelty_metric = total_novelty_score / processed_requests_count

print(f"Novelty@10: {final_novelty_metric:.4f}")

# Метрики для оценки Serendipity в рекомендательных системах

**Serendipity** отражает способность системы рекомендовать неожиданные, но полезные элементы, выходящие за рамки очевидных предпочтений пользователя.  
Измерение сложное, так как требует учета **релевантности** и **неожиданности**. Приведем основные подходы:

---

## 1. **Классическая формула (на основе ожиданий)**
Серендипность = Релевантность × Неожиданность:
$$
\text{Serendipity}(i) = \text{Rel}(i) \times \left(1 - \text{Prob}_{\text{user}}(i)\right)
$$
- `Rel(i)` — релевантность элемента `i` для пользователя (например, оценка или клик).
- `Prob_user(i)` — вероятность того, что пользователь **ожидал** элемент `i` (например, на основе его истории).

---

## 2. **Метрика на основе популярности**
Учитывает редкость рекомендации в общем контексте:
$$
\text{Serendipity}(i) = \text{Rel}(i) \times \left(1 - \text{Popularity}(i)\right)
$$
- `Popularity(i)` — нормированная популярность элемента `i` (например, доля пользователей, взаимодействовавших с ним).

In [ ]:
user_product_purchase_history = (
    train_parts_1_2.filter(pl.col('action_type') == "AT_Purchase")
    .group_by(['user_id', 'product_id'])
    .agg(
        pl.lit(1).alias('has_purchased')
    )
)

total_serendipity_score = 0.0
processed_recommendation_requests = 0

predicts_with_history = catboost_predicts.join(
    user_product_purchase_history,
    on=['user_id', 'product_id'],
    how='left'
).with_columns(
    pl.col('has_purchased').fill_null(0)
).group_by('request_id')

for request_id, recommendations in predicts_with_history:
    top10_recommendations = recommendations.sort('target', descending=True).head(10)

    serendipity_values = (1 - top10_recommendations['has_purchased']) * top10_recommendations['predict']
    average_serendipity = serendipity_values.mean()

    if average_serendipity is not None:
        processed_recommendation_requests += 1
        total_serendipity_score += average_serendipity

final_serendipity_metric = total_serendipity_score / processed_recommendation_requests
print(f"Serendipity@10: {final_serendipity_metric:.4f}")

### Как можно улучшить скор:

- Чистим датасет
- Больше фичей
- Варим фичи более умным способом:
<center><img src="Timesplit2.svg" width="1100" /></center>